# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys2 import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,pacific grove,36.6177,-121.9166,286.29,83,100,4.12,US,1703253227
1,1,isafjordur,66.0755,-23.1240,271.17,74,100,7.20,IS,1703253364
2,2,piripiri,-4.2733,-41.7769,309.02,32,7,4.12,BR,1703253364
3,3,vernon,50.2581,-119.2691,279.04,89,100,1.54,CA,1703253176
4,4,taghazout,30.5426,-9.7112,296.58,34,0,4.07,MA,1703253364


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_df = city_data_df[(city_data_df['Max Temp'] < 310) & (city_data_df['Max Temp'] > 290) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
cleaned_narrowed_df = narrowed_df.dropna()

# Display sample data
cleaned_narrowed_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,taghazout,30.5426,-9.7112,296.58,34,0,4.07,MA,1703253364
47,47,plaza huincul,-38.9260,-69.2086,297.31,15,0,4.19,AR,1703253374
56,56,alice springs,-23.7000,133.8833,298.90,14,0,2.57,AU,1703253375
58,58,valparaiso,-33.0393,-71.6273,293.71,64,0,2.57,CL,1703253091
183,183,callao,-12.0667,-77.1500,296.33,78,0,3.60,PE,1703253117
187,187,conil de la frontera,36.2772,-6.0885,290.87,56,0,1.78,ES,1703253405
242,242,bogande,12.9714,-0.1436,304.84,10,0,3.64,BF,1703253414
248,248,ghat,24.9647,10.1728,290.60,27,0,3.66,LY,1703253166
311,311,mount isa,-20.7333,139.5000,303.02,12,0,3.60,AU,1703253427
400,400,bernardo de irigoyen,-26.2552,-53.6458,302.19,51,0,1.55,AR,1703253448


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_narrowed_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,taghazout,MA,30.5426,-9.7112,34,
47,plaza huincul,AR,-38.9260,-69.2086,15,
56,alice springs,AU,-23.7000,133.8833,14,
58,valparaiso,CL,-33.0393,-71.6273,64,
183,callao,PE,-12.0667,-77.1500,78,
187,conil de la frontera,ES,36.2772,-6.0885,56,
242,bogande,BF,12.9714,-0.1436,10,
248,ghat,LY,24.9647,10.1728,27,
311,mount isa,AU,-20.7333,139.5000,12,
400,bernardo de irigoyen,AR,-26.2552,-53.6458,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address_response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address_response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
taghazout - nearest hotel: Riad Taghazout
plaza huincul - nearest hotel: Antu Malal Aparts & Suites
alice springs - nearest hotel: Aurora Alice Springs
valparaiso - nearest hotel: Ibis Hotel
callao - nearest hotel: Casa Ronald
conil de la frontera - nearest hotel: San Vicente
bogande - nearest hotel: No hotel found
ghat - nearest hotel: فندق تاسيلي
mount isa - nearest hotel: Ibis Styles
bernardo de irigoyen - nearest hotel: Irigoyen
mhamid - nearest hotel: Hotel Kasbah Azalay
vila franca do campo - nearest hotel: Vinha d'Areia Beach Hotel
al ain city - nearest hotel: Al ain sanayya restaurant
jimena de la frontera - nearest hotel: Casa Henrietta
gorom-gorom - nearest hotel: No hotel found
chos malal - nearest hotel: Hospedaje Lemus
al jawf - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
4,taghazout,MA,30.5426,-9.7112,34,Riad Taghazout
47,plaza huincul,AR,-38.9260,-69.2086,15,Antu Malal Aparts & Suites
56,alice springs,AU,-23.7000,133.8833,14,Aurora Alice Springs
58,valparaiso,CL,-33.0393,-71.6273,64,Ibis Hotel
183,callao,PE,-12.0667,-77.1500,78,Casa Ronald
187,conil de la frontera,ES,36.2772,-6.0885,56,San Vicente
242,bogande,BF,12.9714,-0.1436,10,No hotel found
248,ghat,LY,24.9647,10.1728,27,فندق تاسيلي
311,mount isa,AU,-20.7333,139.5000,12,Ibis Styles
400,bernardo de irigoyen,AR,-26.2552,-53.6458,51,Irigoyen


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)